In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob

import matplotlib as mpl

import matplotlib.pyplot as plt

import math as mt

import cartopy.crs as ccrs

import cartopy.feature as cfeature

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

#AO Calculation

In [19]:
# Calculate AO - input std normal latitude wighted 1000mb z
ZData = xr.open_dataset('../../../Data/CMIP6/MIROC6/NDJF/weighted1000Std.nc')
#SLPData = xr.open_dataset('../eraDown/ERA5_2degree_Down/DailyMean/weighted1000.nc')

ZData.zg.values
ZData.zg.values=xr.where(np.isnan(ZData.zg.values),  0.000000000001,ZData.zg.values)
ZData.zg.values


array([[[ 9.99999996e-13,  9.99999996e-13,  9.99999996e-13, ...,
          9.99999996e-13,  9.99999996e-13,  9.99999996e-13],
        [ 9.99999996e-13,  9.99999996e-13,  9.99999996e-13, ...,
          9.99999996e-13,  9.99999996e-13,  9.99999996e-13],
        [ 9.99999996e-13,  9.99999996e-13,  9.99999996e-13, ...,
          9.99999996e-13,  9.99999996e-13,  9.99999996e-13],
        ...,
        [ 3.34836036e-01,  3.32517445e-01,  3.29951912e-01, ...,
          3.40782940e-01,  3.38970542e-01,  3.36901724e-01],
        [ 2.50281960e-01,  2.49075592e-01,  2.47737944e-01, ...,
          2.53706515e-01,  2.52628356e-01,  2.51459450e-01],
        [ 1.50967836e-01,  1.50541946e-01,  1.50096074e-01, ...,
          1.52276322e-01,  1.51847795e-01,  1.51401773e-01]],

       [[ 9.99999996e-13,  9.99999996e-13,  9.99999996e-13, ...,
          9.99999996e-13,  9.99999996e-13,  9.99999996e-13],
        [ 9.99999996e-13,  9.99999996e-13,  9.99999996e-13, ...,
          9.99999996e-13,  9.99999996e

In [20]:


t1=ZData.zg.stack(z=("lat", "lon"))
# fit scaler on training data
zg_PC =PCA().fit_transform(t1)


In [21]:
print(zg_PC[:,0].shape)
AO_ds = xr.Dataset({'AO': (('time'), zg_PC[:,0])}, coords={'time': ZData.time})


norm = StandardScaler().fit(zg_PC[:,0].reshape(-1, 1))
    # transform training data
aoX = norm.transform(zg_PC[:,0].reshape(-1, 1))
aoX.shape

AO_ds = xr.Dataset({'AO': (('time'), aoX[:,0]*-1.0)}, coords={'time': ZData.time})


(19840,)


In [22]:
AO_ds.to_netcdf('MIROC6-AOindex-NDJF-Daily-1980-2014.nc')

In [23]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.max(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(3.5441852, dtype=float32)
Coordinates:
    time     datetime64[ns] 1960-01-12T12:00:00

In [24]:
#AO_ds.AO.where(AO_ds.AO>2.0, drop=True).squeeze()

In [25]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.min(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(-3.8025115, dtype=float32)
Coordinates:
    time     datetime64[ns] 1890-12-01T12:00:00

In [26]:
#AO_ds.AO.where(AO_ds.AO<-2, drop=True).squeeze()

#EU Calculation

In [2]:
zLevData = xr.open_dataset('../../../Data/CMIP6/MIROC6/NDJF/zg_NDJFday_MIROC6_historical_r1i1p1f1_gn_18500101-20141231.nc')

In [3]:
zLevData.time

<xarray.DataArray 'time' (time: 19840)>
array(['1850-01-01T12:00:00.000000000', '1850-01-02T12:00:00.000000000',
       '1850-01-03T12:00:00.000000000', ..., '2014-12-29T12:00:00.000000000',
       '2014-12-30T12:00:00.000000000', '2014-12-31T12:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1850-01-01T12:00:00 ... 2014-12-31T12:00:00
Attributes:
    standard_name:  time
    long_name:      time
    bounds:         time_bnds
    axis:           T

In [4]:
z500=zLevData.zg.sel(plev=50000.0,method='nearest')
z500

<xarray.DataArray 'zg' (time: 19840, lat: 128, lon: 256)>
[650117120 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1850-01-01T12:00:00 ... 2014-12-31T12:00:00
  * lon      (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
  * lat      (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
    plev     float64 5e+04
Attributes:
    standard_name:  geopotential_height
    long_name:      Geopotential Height
    units:          m
    comment:        Geopotential is the sum of the specific gravitational pot...
    original_name:  Z
    cell_methods:   time: mean
    cell_measures:  area: areacella
    history:        2019-09-05T06:53:51Z altered by CMOR: replaced missing va...

In [5]:
z145E40N=z500.sel(lat=40,lon=145,method='nearest')

In [6]:
#z145E40N

In [7]:
z20E55N=z500.sel(lat=55,lon=20,method='nearest')

In [8]:
#z20E55N

In [9]:
z75E55N=z500.sel(lat=55,lon=75,method='nearest')

In [10]:
#z75E55N

In [11]:
#Calculate Anomaly
z145E40N_Anom = z145E40N.groupby("time.dayofyear") - z145E40N.groupby("time.dayofyear").mean("time")
z75E55N_Anom  = z75E55N.groupby("time.dayofyear")  - z75E55N.groupby("time.dayofyear").mean("time")
z20E55N_Anom  = z20E55N.groupby("time.dayofyear")  - z20E55N.groupby("time.dayofyear").mean("time")

In [12]:
# calculate std normal anomalies
z145E40N_AnomStd = z145E40N_Anom.groupby("time.dayofyear")  / z145E40N.groupby("time.dayofyear").std("time")
z75E55N_AnomStd  = z75E55N_Anom.groupby("time.dayofyear")   / z75E55N.groupby("time.dayofyear").std("time")
z20E55N_AnomStd  = z20E55N_Anom.groupby("time.dayofyear")   / z20E55N.groupby("time.dayofyear").std("time")

In [13]:
EUVal = -(1.0/4.0)*z20E55N_AnomStd + (1.0/2.0)*z75E55N_AnomStd - (1.0/4.0)*z145E40N_AnomStd

In [14]:
EUVal

<xarray.DataArray 'zg' (time: 19840)>
array([1.42512542, 1.02113295, 0.68787867, ..., 0.61449647, 0.95087814,
       0.85905198])
Coordinates:
  * time       (time) datetime64[ns] 1850-01-01T12:00:00 ... 2014-12-31T12:00:00
    plev       float64 5e+04
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
    lon        float64 144.8

In [15]:
EU_ds = xr.Dataset({'EU': (('time'), EUVal)}, coords={'time': zLevData.time})

In [16]:
EU_ds.to_netcdf('MIROC6-EUindex-NDJF-Daily-1980-2014.nc')